# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** 

The basic architecture is like this: A pretrained ResNet serves as CNN encoder to extract spatial features from input image. The extracted feature will be the first input to two-layered LSTM, the decoder, to generate the first output word (which is the start word). The rest of sentence will be fed into the decoder in sequence until the max len is reached or the end word is generated. In the training phase, each batch consists of images and captions of the same length. These captions will be fed into the decoder in order along with the embeded spatial features as the first input. This will help train the model in parallel. And a dropout layer is included in the first layer of LSTM which introduces a certain amount of randomness and helps reduce overfitting. In the testing phase, a sampling method will be used in this application, where the output word is fed back into the decoder until the max lengh of sentence is reached or the end word is generated.

In term of hyperparameters' selection, 256 is chosen as the embed size and 512 is selected as the hidden size according to the guidlines by Andrej Karpathy [here](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) where the total number of parameters should be larger than and about the same order of the size of training samples. There are total 414K captions with each with about 10 words on avarge. Plus we also should add the complexity of image features into considerations. A two layered LSTM with 256 as embed size and 512 as hidden size has 11M parameters in total. This should be enough for this application. Note that the parameters of the encoder is not to be trained except its final fc/embed layer.

I choose 10 as the batch size as it is enough to provide a good sample distribution and helps to speed up the weight updating. No vocabulary threshold is set as I think this keeps the integrity of vocab library and the model will be able to handle some rare cases based on word embedding and context in sentences. 3 is chose as the number of epochs as it is enough to produce a good results and the loss decaying is very much slowing down at epoch 3 already.

I read the paper [here](https://arxiv.org/pdf/1411.4555.pdf) before implementing anything in this project and consult it during the process as well.

### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** 

I left the transform as it is because it is good enough to transform the sample images into the format that fits the requirement of ResNet(224x224, normalized in each color channel). More than that, it randomly crops from the resized image, which is slightly larger than 224x224 and perform random horizontal flipping. This is essentially some sort of data augmentation that helps to train a robust model and reduce overfitting.

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** 

The parameters in decoder and the fc/embed layer of encoder are set to be trainable. This is a good choice since all the parameters in decoder (word embedding, LSTM, linear layer) have to be trained in order to understand how to caption an image based on the spatial features. The fc/embed layer in encoder is also trainable since it transform the spatial features into forms that decoder understands. The parameters in CNN/ResNet layers are not to be trained as they are all ready good enough in terms of extracting spatial features from raw images. Setting them trainable will most likely produce poorer results.

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** 

I choose Adam as the optimizer instead of SGD as it adaptively adjust the learning rate and is able to produce a model that is about as good as what SGD could deliver. This is important for a starter model as we don't know exacly the range of learing rate for a SGD that could balance the training time and model performance.

I consult the blog of Andrej Karpathy [here](http://karpathy.github.io/2019/04/25/recipe/) for optimizer selection in this application.

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 10          # batch size
vocab_threshold = None        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 256           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters())

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params=params, lr=3e-4)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...


  0%|          | 0/414113 [00:00<?, ?it/s]

Done (t=0.42s)
creating index...
index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [00:32<00:00, 12892.36it/s]


In [2]:
total = sum([param.nelement() for param in decoder.parameters()])
print("total number of parameters in decoder %d" % total)

total number of parameters in decoder 11333603


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [3]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
         
        if time.time() - old_time > 60:   
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

Epoch [1/3], Step [100/41412], Loss: 4.6395, Perplexity: 103.4956
Epoch [1/3], Step [200/41412], Loss: 4.3535, Perplexity: 77.74991
Epoch [1/3], Step [300/41412], Loss: 4.4092, Perplexity: 82.20227
Epoch [1/3], Step [400/41412], Loss: 4.5191, Perplexity: 91.75084
Epoch [1/3], Step [500/41412], Loss: 4.1842, Perplexity: 65.64149
Epoch [1/3], Step [600/41412], Loss: 3.7552, Perplexity: 42.74143
Epoch [1/3], Step [700/41412], Loss: 4.0753, Perplexity: 58.86895
Epoch [1/3], Step [800/41412], Loss: 3.4370, Perplexity: 31.0927
Epoch [1/3], Step [900/41412], Loss: 3.8826, Perplexity: 48.55038
Epoch [1/3], Step [1000/41412], Loss: 4.4518, Perplexity: 85.7799
Epoch [1/3], Step [1100/41412], Loss: 3.2169, Perplexity: 24.95117
Epoch [1/3], Step [1200/41412], Loss: 3.0506, Perplexity: 21.1288
Epoch [1/3], Step [1300/41412], Loss: 3.6361, Perplexity: 37.94195
Epoch [1/3], Step [1400/41412], Loss: 4.3090, Perplexity: 74.36642
Epoch [1/3], Step [1500/41412], Loss: 3.9286, Perplexity: 50.83737
Epoch [

Epoch [1/3], Step [24600/41412], Loss: 2.0707, Perplexity: 7.93042
Epoch [1/3], Step [24700/41412], Loss: 2.2346, Perplexity: 9.34297
Epoch [1/3], Step [24800/41412], Loss: 2.4616, Perplexity: 11.7239
Epoch [1/3], Step [24900/41412], Loss: 2.5632, Perplexity: 12.9776
Epoch [1/3], Step [25000/41412], Loss: 1.9236, Perplexity: 6.84532
Epoch [1/3], Step [25100/41412], Loss: 3.1364, Perplexity: 23.0217
Epoch [1/3], Step [25200/41412], Loss: 2.4249, Perplexity: 11.3013
Epoch [1/3], Step [25300/41412], Loss: 2.6546, Perplexity: 14.2199
Epoch [1/3], Step [25400/41412], Loss: 2.4586, Perplexity: 11.6884
Epoch [1/3], Step [25500/41412], Loss: 2.6356, Perplexity: 13.9516
Epoch [1/3], Step [25600/41412], Loss: 2.2970, Perplexity: 9.94487
Epoch [1/3], Step [25700/41412], Loss: 2.8171, Perplexity: 16.7285
Epoch [1/3], Step [25800/41412], Loss: 1.8551, Perplexity: 6.39204
Epoch [1/3], Step [25900/41412], Loss: 2.8438, Perplexity: 17.1815
Epoch [1/3], Step [26000/41412], Loss: 2.2533, Perplexity: 9.5

Epoch [2/3], Step [7700/41412], Loss: 1.8731, Perplexity: 6.50857
Epoch [2/3], Step [7800/41412], Loss: 1.5257, Perplexity: 4.59855
Epoch [2/3], Step [7900/41412], Loss: 2.1547, Perplexity: 8.62519
Epoch [2/3], Step [8000/41412], Loss: 2.6250, Perplexity: 13.8052
Epoch [2/3], Step [8100/41412], Loss: 2.7008, Perplexity: 14.8922
Epoch [2/3], Step [8200/41412], Loss: 2.9468, Perplexity: 19.0443
Epoch [2/3], Step [8300/41412], Loss: 1.9471, Perplexity: 7.00827
Epoch [2/3], Step [8400/41412], Loss: 2.1933, Perplexity: 8.96499
Epoch [2/3], Step [8500/41412], Loss: 2.3574, Perplexity: 10.5637
Epoch [2/3], Step [8600/41412], Loss: 3.2853, Perplexity: 26.7162
Epoch [2/3], Step [8700/41412], Loss: 2.5661, Perplexity: 13.0145
Epoch [2/3], Step [8800/41412], Loss: 1.9627, Perplexity: 7.11862
Epoch [2/3], Step [8900/41412], Loss: 2.0088, Perplexity: 7.45428
Epoch [2/3], Step [9000/41412], Loss: 2.5520, Perplexity: 12.8333
Epoch [2/3], Step [9100/41412], Loss: 2.5942, Perplexity: 13.3861
Epoch [2/3

Epoch [2/3], Step [32100/41412], Loss: 2.3541, Perplexity: 10.5281
Epoch [2/3], Step [32200/41412], Loss: 2.4173, Perplexity: 11.2158
Epoch [2/3], Step [32300/41412], Loss: 2.2219, Perplexity: 9.22475
Epoch [2/3], Step [32400/41412], Loss: 2.3735, Perplexity: 10.7344
Epoch [2/3], Step [32500/41412], Loss: 1.8743, Perplexity: 6.51631
Epoch [2/3], Step [32600/41412], Loss: 2.1107, Perplexity: 8.25420
Epoch [2/3], Step [32700/41412], Loss: 2.6342, Perplexity: 13.9317
Epoch [2/3], Step [32800/41412], Loss: 1.8955, Perplexity: 6.65561
Epoch [2/3], Step [32900/41412], Loss: 1.8228, Perplexity: 6.18942
Epoch [2/3], Step [33000/41412], Loss: 3.4056, Perplexity: 30.1329
Epoch [2/3], Step [33100/41412], Loss: 2.3256, Perplexity: 10.2333
Epoch [2/3], Step [33200/41412], Loss: 1.8668, Perplexity: 6.46744
Epoch [2/3], Step [33300/41412], Loss: 2.6899, Perplexity: 14.7295
Epoch [2/3], Step [33400/41412], Loss: 1.9369, Perplexity: 6.93732
Epoch [2/3], Step [33500/41412], Loss: 1.9581, Perplexity: 7.0

Epoch [3/3], Step [15200/41412], Loss: 2.1908, Perplexity: 8.94244
Epoch [3/3], Step [15300/41412], Loss: 2.3293, Perplexity: 10.2704
Epoch [3/3], Step [15400/41412], Loss: 2.7241, Perplexity: 15.2425
Epoch [3/3], Step [15500/41412], Loss: 1.9208, Perplexity: 6.82628
Epoch [3/3], Step [15600/41412], Loss: 1.8256, Perplexity: 6.20672
Epoch [3/3], Step [15700/41412], Loss: 1.9256, Perplexity: 6.85924
Epoch [3/3], Step [15800/41412], Loss: 1.7285, Perplexity: 5.63217
Epoch [3/3], Step [15900/41412], Loss: 3.0017, Perplexity: 20.1199
Epoch [3/3], Step [16000/41412], Loss: 1.8474, Perplexity: 6.34361
Epoch [3/3], Step [16100/41412], Loss: 2.1996, Perplexity: 9.02180
Epoch [3/3], Step [16200/41412], Loss: 2.5365, Perplexity: 12.6360
Epoch [3/3], Step [16300/41412], Loss: 2.1599, Perplexity: 8.67036
Epoch [3/3], Step [16400/41412], Loss: 2.2810, Perplexity: 9.78692
Epoch [3/3], Step [16500/41412], Loss: 2.2681, Perplexity: 9.66104
Epoch [3/3], Step [16600/41412], Loss: 2.0235, Perplexity: 7.5

Epoch [3/3], Step [39600/41412], Loss: 1.9967, Perplexity: 7.36479
Epoch [3/3], Step [39700/41412], Loss: 2.0269, Perplexity: 7.59052
Epoch [3/3], Step [39800/41412], Loss: 1.5355, Perplexity: 4.64388
Epoch [3/3], Step [39900/41412], Loss: 1.7413, Perplexity: 5.70489
Epoch [3/3], Step [40000/41412], Loss: 1.8344, Perplexity: 6.26149
Epoch [3/3], Step [40100/41412], Loss: 2.0430, Perplexity: 7.71382
Epoch [3/3], Step [40200/41412], Loss: 2.2466, Perplexity: 9.45531
Epoch [3/3], Step [40300/41412], Loss: 2.7150, Perplexity: 15.1043
Epoch [3/3], Step [40400/41412], Loss: 1.6652, Perplexity: 5.28696
Epoch [3/3], Step [40500/41412], Loss: 1.9706, Perplexity: 7.17528
Epoch [3/3], Step [40600/41412], Loss: 2.1449, Perplexity: 8.54102
Epoch [3/3], Step [40700/41412], Loss: 2.3913, Perplexity: 10.9277
Epoch [3/3], Step [40800/41412], Loss: 2.0429, Perplexity: 7.71300
Epoch [3/3], Step [40900/41412], Loss: 1.5915, Perplexity: 4.91116
Epoch [3/3], Step [41000/41412], Loss: 2.1966, Perplexity: 8.9

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [4]:
# (Optional) TODO: Validate your model.